<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"> <img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png"/> </a> <br/> This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"> Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License. </a>

#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# **Spark Tutorial: Learning Apache Spark**

This tutorial will teach you how to use [Apache Spark](http://spark.apache.org/), a framework for large-scale data processing, within a notebook. Many traditional frameworks were designed to be run on a single computer.  However, many datasets today are too large to be stored on a single computer, and even when a dataset can be stored on one computer (such as the datasets in this tutorial), the dataset can often be processed much more quickly using multiple computers.

Spark has efficient implementations of a number of transformations and actions that can be composed together to perform data processing and analysis.  Spark excels at distributing these operations across a cluster while abstracting away many of the underlying implementation details.  Spark has been designed with a focus on scalability and efficiency.  With Spark you can begin developing your solution on your laptop, using a small dataset, and then use that same code to process terabytes or even petabytes across a distributed cluster.

**During this tutorial we will cover:**

* *Part 1:* Basic notebook usage and [Python](https://docs.python.org/2/) integration
* *Part 2:* An introduction to using [Apache Spark](https://spark.apache.org/) with the [PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module) running in a notebook
* *Part 3:* Using DataFrames and chaining together transformations and actions
* *Part 4*: Python Lambda functions and User Defined Functions
* *Part 5:* Additional DataFrame actions
* *Part 6:* Additional DataFrame transformations
* *Part 7:* Caching DataFrames and storage options
* *Part 8:* Debugging Spark applications and lazy evaluation

The following transformations will be covered:
* `select()`, `filter()`, `distinct()`, `dropDuplicates()`, `orderBy()`, `groupBy()`

The following actions will be covered:
* `first()`, `take()`, `count()`, `collect()`, `show()`

Also covered:
* `cache()`, `unpersist()`

Note that, for reference, you can look up the details of these methods in the [Spark's PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)

## **Part 1: Basic notebook usage and [Python](https://docs.python.org/2/) integration **

### (1a) Notebook usage

A notebook is comprised of a linear sequence of cells.  These cells can contain either markdown or code, but we won't mix both in one cell.  When a markdown cell is executed it renders formatted text, images, and links just like HTML in a normal webpage.  The text you are reading right now is part of a markdown cell.  Python code cells allow you to execute arbitrary Python commands just like in any Python shell. Place your cursor inside the cell below, and press "Shift" + "Enter" to execute the code and advance to the next cell.  You can also press "Ctrl" + "Enter" to execute the code and remain in the cell.  These commands work the same in both markdown and code cells.

In [5]:
# This is a Python cell. You can run normal Python code here...
print 'The sum of 1 and 1 is {0}'.format(1+1)

The sum of 1 and 1 is 2

In [6]:
# Here is another Python cell, this time with a variable (x) declaration and an if statement:
x = 42
if x > 40:
    print 'The sum of 1 and 2 is {0}'.format(1+2)

The sum of 1 and 2 is 3

### (1b) Notebook state

As you work through a notebook it is important that you run all of the code cells.  The notebook is stateful, which means that variables and their values are retained until the notebook is detached (in Databricks) or the kernel is restarted (in Jupyter notebooks).  If you do not run all of the code cells as you proceed through the notebook, your variables will not be properly initialized and later code might fail.  You will also need to rerun any cells that you have modified in order for the changes to be available to other cells.

In [8]:
# This cell relies on x being defined already.
# If we didn't run the cells from part (1a) this code would fail.
print x * 2

84

### (1c) Library imports

We can import standard Python libraries ([modules](https://docs.python.org/2/tutorial/modules.html)) the usual way.  An `import` statement will import the specified module.  In this tutorial and future labs, we will provide any imports that are necessary.

In [10]:
# Import the regular expression library
import re
m = re.search('(?<=abc)def', 'abcdef')
m.group(0)

Out[ 4 ]: 'def'

In [11]:
# Import the datetime library
import datetime
print 'This was last run on: {0}'.format(datetime.datetime.now())

This was last run on: 2018-10-11 16:30:58.398389

##  **Part 2: An introduction to using [Apache Spark](https://spark.apache.org/) with the [PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module) running in a notebook**

### Spark Context

In Spark, communication occurs between a driver and executors.  The driver has Spark jobs that it needs to run and these jobs are split into tasks that are submitted to the executors for completion.  The results from these tasks are delivered back to the driver.

In part 1, we saw that normal Python code can be executed via cells. **When using Databricks this code gets executed in the Spark driver's Java Virtual Machine (JVM) and not in an executor's JVM, and when using an Jupyter notebook it is executed within the kernel associated with the notebook. Since no Spark functionality is actually being used, no tasks are launched on the executors.**

In order to use Spark and its DataFrame API we will need to use a `SQLContext`.  When running Spark, you start a new Spark application by creating a [SparkContext](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext). You can then create a [SQLContext](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext) from the `SparkContext`. When the `SparkContext` is created, it asks the master for some cores to use to do work.  The master sets these cores aside just for you; they won't be used for other applications. **When using Databricks, both a `SparkContext` and a `SQLContext` are created for you automatically. `sc` is your `SparkContext`, and `sqlContext` is your `SQLContext`.**

### (2a) Example Cluster
The diagram shows an example cluster, where the slots allocated for an application are outlined in purple. (Note: We're using the term _slots_ here to indicate threads available to perform parallel work for Spark.
Spark documentation often refers to these threads as _cores_, which is a confusing term, as the number of slots available on a particular machine does not necessarily have any relationship to the number of physical CPU
cores on that machine.)

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-2a.png" style="height: 800px;float: right"/>

You can view the details of your Spark application in the Spark web UI.  The web UI is accessible in Databricks by going to "Clusters" and then clicking on the "Spark UI" link for your cluster.  In the web UI, under the "Jobs" tab, you can see a list of jobs that have been scheduled or run.  It's likely there isn't any thing interesting here yet because we haven't run any jobs, but we'll return to this page later.

At a high level, every Spark application consists of a driver program that launches various parallel operations on executor Java Virtual Machines (JVMs) running either in a cluster or locally on the same machine. In Databricks, "Databricks Shell" is the driver program.  When running locally, `pyspark` is the driver program. In all cases, this driver program contains the main loop for the program and creates distributed datasets on the cluster, then applies operations (transformations & actions) to those datasets.
Driver programs access Spark through a SparkContext object, which represents a connection to a computing cluster. **A Spark SQL context object (`sqlContext`) is the main entry point for Spark DataFrame and SQL functionality.** A `SQLContext` can be used to create DataFrames, which allows you to direct the operations on your data.

Try printing out `sqlContext` to see its type.

In [15]:
# Display the type of the Spark sqlContext
type(sqlContext)

Out[ 6 ]: pyspark.sql.context.HiveContext

Note that the type is `HiveContext`. This means we're working with **a version of Spark that has Hive support.** Compiling Spark with Hive support is a good idea, even if you don't have a Hive metastore. As the
[Spark Programming Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html#starting-point-sqlcontext) states, a `HiveContext` "provides a superset of the functionality provided by the basic `SQLContext`. Additional features include the ability to write queries using the more complete HiveQL parser, access to Hive UDFs [user-defined functions], and the ability to read data from Hive tables. To use a `HiveContext`, you do not need to have an existing Hive setup, and all of the data sources available to a `SQLContext` are still available."

### (2b) SparkContext attributes

You can use Python's [dir()](https://docs.python.org/2/library/functions.html?highlight=dir#dir) function to get a list of all the attributes (including methods) accessible through the `sqlContext` object.

In [18]:
# List sqlContext's attributes
dir(sqlContext)

Out[ 7 ]: 
['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_createForTesting',
 '_inferSchema',
 '_instantiatedContext',
 '_jsc',
 '_jsqlContext',
 '_jvm',
 '_sc',
 '_ssql_ctx',
 'cacheTable',
 'clearCache',
 'createDataFrame',
 'createExternalTable',
 'dropTempTable',
 'getConf',
 'getOrCreate',
 'newSession',
 'range',
 'read',
 'readStream',
 'refreshTable',
 'registerDataFrameAsTable',
 'registerFunction',
 'registerJavaFunction',
 'setConf',
 'sparkSession',
 'sql',
 'streams',
 'table',
 'tableNames',
 'tables',
 'udf',
 'uncacheTable']

### (2c) Getting help

Alternatively, you can use Python's [help()](https://docs.python.org/2/library/functions.html?highlight=help#help) function to get an easier to read list of all the attributes, including examples, that the `sqlContext` object has.

In [20]:
# Use help to obtain more detailed information
help(sqlContext)

Help on HiveContext in module pyspark.sql.context object:

class HiveContext(SQLContext)
 A variant of Spark SQL that integrates with data stored in Hive.
 
 Configuration for Hive is read from ``hive-site.xml`` on the classpath.
 It supports running both SQL and HiveQL commands.
 
 :param sparkContext: The SparkContext to wrap.
 :param jhiveContext: An optional JVM Scala HiveContext. If set, we do not instantiate a new
 :class:`HiveContext` in the JVM, instead we make all calls to this object.
 
 .. note:: Deprecated in 2.0.0. Use SparkSession.builder.enableHiveSupport().getOrCreate().
 
 Method resolution order:
 HiveContext
 SQLContext
 __builtin__.object
 
 Methods defined here:
 
 __init__(self, sparkContext, jhiveContext=None)
 
 refreshTable(self, tableName)
 Invalidate and refresh all the cached the metadata of the given
 table. For performance reasons, Spark SQL or the external data source
 library it uses might cache certain metadata about a table, such as the
 location of blocks. When those change outside of Spark SQL, users should
 call this function to invalidate the cache.
 
 ----------------------------------------------------------------------
 Methods inherited from SQLContext:
 
 cacheTable(self, tableName)
 Caches the specified table in-memory.
 
 .. versionadded:: 1.0
 
 clearCache(self)
 Removes all cached tables from the in-memory cache.
 
 .. versionadded:: 1.3
 
 createDataFrame(self, data, schema=None, samplingRatio=None, verifySchema=True)
 Creates a :class:`DataFrame` from an :class:`RDD`, a list or a :class:`pandas.DataFrame`.
 
 When ``schema`` is a list of column names, the type of each column
 will be inferred from ``data``.
 
 When ``schema`` is ``None``, it will try to infer the schema (column names and types)
 from ``data``, which should be an RDD of :class:`Row`,
 or :class:`namedtuple`, or :class:`dict`.
 
 When ``schema`` is :class:`pyspark.sql.types.DataType` or a datatype string it must match
 the real data, or an exception will be thrown at runtime. If the given schema is not
 :class:`pyspark.sql.types.StructType`, it will be wrapped into a
 :class:`pyspark.sql.types.StructType` as its only field, and the field name will be "value",
 each record will also be wrapped into a tuple, which can be converted to row later.
 
 If schema inference is needed, ``samplingRatio`` is used to determined the ratio of
 rows used for schema inference. The first row will be used if ``samplingRatio`` is ``None``.
 
 :param data: an RDD of any kind of SQL data representation(e.g. :class:`Row`,
 :class:`tuple`, ``int``, ``boolean``, etc.), or :class:`list`, or
 :class:`pandas.DataFrame`.
 :param schema: a :class:`pyspark.sql.types.DataType` or a datatype string or a list of
 column names, default is None. The data type string format equals to
 :class:`pyspark.sql.types.DataType.simpleString`, except that top level struct type can
 omit the ``struct<>`` and atomic types use ``typeName()`` as their format, e.g. use
 ``byte`` instead of ``tinyint`` for :class:`pyspark.sql.types.ByteType`.
 We can also use ``int`` as a short name for :class:`pyspark.sql.types.IntegerType`.
 :param samplingRatio: the sample ratio of rows used for inferring
 :param verifySchema: verify data types of every row against schema.
 :return: :class:`DataFrame`
 
 .. versionchanged:: 2.0
 The ``schema`` parameter can be a :class:`pyspark.sql.types.DataType` or a
 datatype string after 2.0.
 If it's not a :class:`pyspark.sql.types.StructType`, it will be wrapped into a
 :class:`pyspark.sql.types.StructType` and each record will also be wrapped into a tuple.
 
 .. versionchanged:: 2.1
 Added verifySchema.
 
 >>> l = [('Alice', 1)]
 >>> sqlContext.createDataFrame(l).collect()
 [Row(_1=u'Alice', _2=1)]
 >>> sqlContext.createDataFrame(l, ['name', 'age']).collect()
 [Row(name=u'Alice', age=1)]
 
 >>> d = [{'name': 'Alice', 'age': 1}]
 >>> sqlContext.createDataFrame(d).collect()
 [Row(age=1, name=u'Alice')]
 
 >>> rdd = sc.parallelize(l)
 >>> sql

Outside of `pyspark` or a notebook, `SQLContext` is created from the lower-level `SparkContext`, which is usually used to create Resilient Distributed Datasets (RDDs). **An RDD is the way Spark actually represents data internally;** DataFrames are actually implemented in terms of RDDs.

While you can interact directly with RDDs, DataFrames are preferred. They're generally faster, and they perform the same no matter what language (Python, R, Scala or Java) you use with Spark.

In this course, we'll be using DataFrames, so we won't be interacting directly with the Spark Context object very much. However, it's worth knowing that **inside `pyspark` or a notebook, you already have an existing `SparkContext` in the `sc` variable**. One simple thing we can do with `sc` is check the version of Spark we're using:

In [22]:
# After reading the help we've decided we want to use sc.version to see what version of Spark we are running.
# Note that the lower case u in front of a string means it's an unicode string. It's only the encoding, and therefore is no harm at all.
sc.version

Out[ 9 ]: u'2.3.1'

In [23]:
# Help can be used on any Python object
help(map)

Help on built-in function map in module __builtin__:

map(...)
 map(function, sequence[, sequence, ...]) -> list
 
 Return a list of the results of applying the function to the items of
 the argument sequence(s). If more than one sequence is given, the
 function is called with an argument list consisting of the corresponding
 item of each sequence, substituting None for missing values when not all
 sequences have the same length. If the function is None, return a list of
 the items of the sequence (or a list of tuples if more than one sequence).

## **Part 3: Using DataFrames and chaining together transformations and actions**

### Working with your first DataFrames

In Spark, we first create a base [DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame). We can then apply one or more transformations to that base DataFrame. **A DataFrame is immutable, so once it is created, it cannot be changed.** As a result, each transformation creates a new DataFrame. Finally, we can apply one or more actions to the DataFrames.

> Note that Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.

We will perform several exercises to obtain a better understanding of DataFrames:
* Create a Python collection of 10,000 integers
* Create a Spark DataFrame from that collection
* Subtract one from each value using `map`
* Perform action `collect` to view results
* Perform action `count` to view counts
* Apply transformation `filter` and view results with `collect`
* Learn about lambda functions
* Explore how lazy evaluation works and the debugging challenges that it introduces

**A DataFrame consists of a series of `Row` objects**; each `Row` object has a set of named columns. You can think of a DataFrame as modeling a table, though the data source being processed does not have to be a table.

More formally, a DataFrame must have a _schema_, which means it must consist of columns, each of which has a _name_ and a _type_. Some data sources have schemas built into them. Examples include RDBMS databases, Parquet files, and NoSQL databases like Cassandra. Other data sources don't have computer-readable schemas, but you can often apply a schema programmatically.

### (3a) Create a Python collection of 10,000 people

We will use a third-party Python testing library called [fake-factory](https://pypi.python.org/pypi/fake-factory/0.5.3) to create a collection of fake person records.

In [27]:
from faker import Faker 
# solution to the error: https://forums.databricks.com/questions/10790/i-am-tring-to-use-faker-python-lib-but-i-am-keep-g.html
fake = Faker()
fake.seed(4321)

We're going to use this factory to create a collection of randomly generated people records. In the next section, we'll turn that collection into a DataFrame. We'll use a Python tuple to help us define the Spark DataFrame schema. There are other ways to define schemas, though; see
the Spark Programming Guide's discussion of [schema inference](http://spark.apache.org/docs/latest/sql-programming-guide.html#inferring-the-schema-using-reflection) for more information. (For instance,
we could also use a Python `namedtuple` or a Spark `Row` object.)

In [29]:
# Each entry consists of last_name, first_name, ssn, job, and age (at least 1)
from pyspark.sql import Row
def fake_entry():
  name = fake.name().split()
  return (name[1], name[0], fake.ssn(), fake.job(), abs(2016 - fake.date_time().year) + 1)

In [30]:
# Create a helper function to call a function repeatedly
def repeat(times, func, *args, **kwargs):
    for _ in xrange(times):
        yield func(*args, **kwargs)

In [31]:
data = list(repeat(10000, fake_entry))

`data` is just a normal Python list, containing Python tuples objects. Let's look at the first item in the list:

In [33]:
data[0]

Out[ 15 ]: (u'Brown', u'Jason', u'160-37-9051', 'Aid worker', 38)

We can check the size of the list using the Python `len()` function.

In [35]:
len(data)

Out[ 16 ]: 10000

### (3b) Distributed data and using a collection to create a DataFrame

In Spark, datasets are represented as a list of entries, where the list is broken up into many different partitions that are each stored on a different machine.  Each partition holds a unique subset of the entries in the list.  Spark calls datasets that it stores "Resilient Distributed Datasets" (RDDs). Even **DataFrames are ultimately represented as RDDs**, with additional meta-data.

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3b.png" style="width: 900px; float: right; margin: 5px"/>

One of the defining features of Spark, compared to other data analytics frameworks (e.g., Hadoop), is that it stores data in memory rather than on disk.  This allows Spark applications to run much more quickly, because they are not slowed down by needing to read data from disk.
The figure to the right illustrates how Spark breaks a list of data entries into partitions that are each stored in memory on a worker.

To create the DataFrame, we'll use `sqlContext.createDataFrame()`, and we'll pass our array of data in as an argument to that function. Spark will create a new set of input data based on data that is passed in.  **A DataFrame requires a _schema_, which is a list of columns, where each column has a name and a type.** Our list of data has elements with types (mostly strings, but one integer). We'll supply the rest of the schema and the column names as the second argument to `createDataFrame()`.

Let's view the help for `createDataFrame()`.

In [38]:
help(sqlContext.createDataFrame)

Help on method createDataFrame in module pyspark.sql.context:

createDataFrame(self, data, schema=None, samplingRatio=None, verifySchema=True) method of pyspark.sql.context.HiveContext instance
 Creates a :class:`DataFrame` from an :class:`RDD`, a list or a :class:`pandas.DataFrame`.
 
 When ``schema`` is a list of column names, the type of each column
 will be inferred from ``data``.
 
 When ``schema`` is ``None``, it will try to infer the schema (column names and types)
 from ``data``, which should be an RDD of :class:`Row`,
 or :class:`namedtuple`, or :class:`dict`.
 
 When ``schema`` is :class:`pyspark.sql.types.DataType` or a datatype string it must match
 the real data, or an exception will be thrown at runtime. If the given schema is not
 :class:`pyspark.sql.types.StructType`, it will be wrapped into a
 :class:`pyspark.sql.types.StructType` as its only field, and the field name will be "value",
 each record will also be wrapped into a tuple, which can be converted to row later.
 
 If schema inference is needed, ``samplingRatio`` is used to determined the ratio of
 rows used for schema inference. The first row will be used if ``samplingRatio`` is ``None``.
 
 :param data: an RDD of any kind of SQL data representation(e.g. :class:`Row`,
 :class:`tuple`, ``int``, ``boolean``, etc.), or :class:`list`, or
 :class:`pandas.DataFrame`.
 :param schema: a :class:`pyspark.sql.types.DataType` or a datatype string or a list of
 column names, default is None. The data type string format equals to
 :class:`pyspark.sql.types.DataType.simpleString`, except that top level struct type can
 omit the ``struct<>`` and atomic types use ``typeName()`` as their format, e.g. use
 ``byte`` instead of ``tinyint`` for :class:`pyspark.sql.types.ByteType`.
 We can also use ``int`` as a short name for :class:`pyspark.sql.types.IntegerType`.
 :param samplingRatio: the sample ratio of rows used for inferring
 :param verifySchema: verify data types of every row against schema.
 :return: :class:`DataFrame`
 
 .. versionchanged:: 2.0
 The ``schema`` parameter can be a :class:`pyspark.sql.types.DataType` or a
 datatype string after 2.0.
 If it's not a :class:`pyspark.sql.types.StructType`, it will be wrapped into a
 :class:`pyspark.sql.types.StructType` and each record will also be wrapped into a tuple.
 
 .. versionchanged:: 2.1
 Added verifySchema.
 
 >>> l = [('Alice', 1)]
 >>> sqlContext.createDataFrame(l).collect()
 [Row(_1=u'Alice', _2=1)]
 >>> sqlContext.createDataFrame(l, ['name', 'age']).collect()
 [Row(name=u'Alice', age=1)]
 
 >>> d = [{'name': 'Alice', 'age': 1}]
 >>> sqlContext.createDataFrame(d).collect()
 [Row(age=1, name=u'Alice')]
 
 >>> rdd = sc.parallelize(l)
 >>> sqlContext.createDataFrame(rdd).collect()
 [Row(_1=u'Alice', _2=1)]
 >>> df = sqlContext.createDataFrame(rdd, ['name', 'age'])
 >>> df.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> from pyspark.sql import Row
 >>> Person = Row('name', 'age')
 >>> person = rdd.map(lambda r: Person(*r))
 >>> df2 = sqlContext.createDataFrame(person)
 >>> df2.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> from pyspark.sql.types import *
 >>> schema = StructType([
 ... StructField("name", StringType(), True),
 ... StructField("age", IntegerType(), True)])
 >>> df3 = sqlContext.createDataFrame(rdd, schema)
 >>> df3.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> sqlContext.createDataFrame(df.toPandas()).collect() # doctest: +SKIP
 [Row(name=u'Alice', age=1)]
 >>> sqlContext.createDataFrame(pandas.DataFrame([[1, 2]])).collect() # doctest: +SKIP
 [Row(0=1, 1=2)]
 
 >>> sqlContext.createDataFrame(rdd, "a: string, b: int").collect()
 [Row(a=u'Alice', b=1)]
 >>> rdd = rdd.map(lambda row: row[1])
 >>> sqlContext.createDataFrame(rdd, "int").collect()
 [Row(value=1)]
 >>> sqlContext.createDataFrame(rdd, "boolean").collect() # doctest: +IGNORE_EXCEPTION_DETAIL
 Traceback (most recent call last):
 ...
 Py4JJavaError: ...
 
 .. versionadded:: 1.3

In [39]:
dataDF = sqlContext.createDataFrame(data, ('last_name', 'first_name', 'ssn', 'occupation', 'age'))

Let's see what type `sqlContext.createDataFrame()` returned.

In [41]:
print 'type of dataDF: {0}'.format(type(dataDF))

type of dataDF: <class 'pyspark.sql.dataframe.DataFrame'>

Let's take a look at the DataFrame's schema and some of its rows.

In [43]:
dataDF.printSchema()

root
-- last_name: string (nullable = true)
-- first_name: string (nullable = true)
-- ssn: string (nullable = true)
-- occupation: string (nullable = true)
-- age: long (nullable = true)

We can register the newly created DataFrame as a named table, using the `registerDataFrameAsTable()` method.

In [45]:
sqlContext.registerDataFrameAsTable(dataDF, 'dataframe')

What methods can we call on this DataFrame?

In [47]:
help(dataDF)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(__builtin__.object)
 A distributed collection of data grouped into named columns.
 
 A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 and can be created using various functions in :class:`SparkSession`::
 
 people = spark.read.parquet("...")
 
 Once created, it can be manipulated using the various domain-specific-language
 (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 
 To select a column from the data frame, use the apply method::
 
 ageCol = people.age
 
 A more concrete example::
 
 # To create DataFrame using SparkSession
 people = spark.read.parquet("...")
 department = spark.read.parquet("...")
 
 people.filter(people.age > 30).join(department, people.deptId == department.id) \
 .groupBy(department.name, "gender").agg({"salary": "avg", "age": "max"})
 
 .. versionadded:: 1.3
 
 Methods defined here:
 
 __getattr__(self, name)
 Returns the :class:`Column` denoted by ``name``.
 
 >>> df.select(df.age).collect()
 [Row(age=2), Row(age=5)]
 
 .. versionadded:: 1.3
 
 __getitem__(self, item)
 Returns the column as a :class:`Column`.
 
 >>> df.select(df['age']).collect()
 [Row(age=2), Row(age=5)]
 >>> df[ ["name", "age"]].collect()
 [Row(name=u'Alice', age=2), Row(name=u'Bob', age=5)]
 >>> df[ df.age > 3 ].collect()
 [Row(age=5, name=u'Bob')]
 >>> df[df[0] > 3].collect()
 [Row(age=5, name=u'Bob')]
 
 .. versionadded:: 1.3
 
 __init__(self, jdf, sql_ctx)
 
 __repr__(self)
 
 agg(self, *exprs)
 Aggregate on the entire :class:`DataFrame` without groups
 (shorthand for ``df.groupBy.agg()``).
 
 >>> df.agg({"age": "max"}).collect()
 [Row(max(age)=5)]
 >>> from pyspark.sql import functions as F
 >>> df.agg(F.min(df.age)).collect()
 [Row(min(age)=2)]
 
 .. versionadded:: 1.3
 
 alias(self, alias)
 Returns a new :class:`DataFrame` with an alias set.
 
 >>> from pyspark.sql.functions import *
 >>> df_as1 = df.alias("df_as1")
 >>> df_as2 = df.alias("df_as2")
 >>> joined_df = df_as1.join(df_as2, col("df_as1.name") == col("df_as2.name"), 'inner')
 >>> joined_df.select("df_as1.name", "df_as2.name", "df_as2.age").collect()
 [Row(name=u'Bob', name=u'Bob', age=5), Row(name=u'Alice', name=u'Alice', age=2)]
 
 .. versionadded:: 1.3
 
 approxQuantile(self, col, probabilities, relativeError)
 Calculates the approximate quantiles of numerical columns of a
 DataFrame.
 
 The result of this algorithm has the following deterministic bound:
 If the DataFrame has N elements and if we request the quantile at
 probability `p` up to error `err`, then the algorithm will return
 a sample `x` from the DataFrame so that the *exact* rank of `x` is
 close to (p * N). More precisely,
 
 floor((p - err) * N) <= rank(x) <= ceil((p + err) * N).
 
 This method implements a variation of the Greenwald-Khanna
 algorithm (with some speed optimizations). The algorithm was first
 present in [[http://dx.doi.org/10.1145/375663.375670
 Space-efficient Online Computation of Quantile Summaries]]
 by Greenwald and Khanna.
 
 Note that null values will be ignored in numerical columns before calculation.
 For columns only containing null values, an empty list is returned.
 
 :param col: str, list.
 Can be a single column name, or a list of names for multiple columns.
 :param probabilities: a list of quantile probabilities
 Each number must belong to [0, 1].
 For example 0 is the minimum, 0.5 is the median, 1 is the maximum.
 :param relativeError: The relative target precision to achieve
 (>= 0). If set to zero, the exact quantiles are computed, which
 could be very expensive. Note that values greater than 1 are
 accepted but give the same result as 1.
 :return: the approximate quantiles at the given probabilities. If
 the input `col` is a string, the output is a list of floats. If the
 input `col` is a list or tuple of strings, the output is also a
 list, but each element in it is a list of floats, i.e., the output
 is a list of list of floats.
 
 .. versionchanged

How many partitions will the DataFrame be split into?

In [49]:
dataDF.rdd.getNumPartitions()

Out[ 23 ]: 8

###### A note about DataFrames and queries

When you use DataFrames or Spark SQL, you are building up a _query plan_. **Each transformation you apply to a DataFrame adds some information to the query plan. When you finally call an action, which triggers execution of your Spark job**, several things happen:

1. Spark's Catalyst optimizer analyzes the query plan (called an _unoptimized logical query plan_) and attempts to optimize it. Optimizations include (but aren't limited to) rearranging and combining `filter()` operations for efficiency, converting `Decimal` operations to more efficient long integer operations, and pushing some operations down into the data source (e.g., a `filter()` operation might be translated to a SQL `WHERE` clause, if the data source is a traditional SQL RDBMS). The result of this optimization phase is an _optimized logical plan_.
2. Once Catalyst has an optimized logical plan, it then constructs multiple _physical_ plans from it. Specifically, it implements the query in terms of lower level Spark RDD operations.
3. Catalyst chooses which physical plan to use via _cost optimization_. That is, it determines which physical plan is the most efficient (or least expensive), and uses that one.
4. Finally, once the physical RDD execution plan is established, Spark actually executes the job.

You can examine the query plan using the `explain()` function on a DataFrame. By default, `explain()` only shows you the final physical plan; however, if you pass it an argument of `True`, it will show you all phases.

(If you want to take a deeper dive into how Catalyst optimizes DataFrame queries, this blog post, while a little old, is an excellent overview: [Deep Dive into Spark SQL's Catalyst Optimizer](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html).)

Let's add a couple transformations to our DataFrame and look at the query plan on the resulting transformed DataFrame. Don't be too concerned if it looks like gibberish. As you gain more experience with Apache Spark, you'll begin to be able to use `explain()` to help you understand more about your DataFrame operations.

In [51]:
newDF = dataDF.distinct().select('*')
newDF.explain(True)

== Parsed Logical Plan ==
'Project [*]
+- AnalysisBarrier
 +- Deduplicate [first_name#597, age#600L, last_name#596, occupation#599, ssn#598]
 +- LogicalRDD [last_name#596, first_name#597, ssn#598, occupation#599, age#600L], false

== Analyzed Logical Plan ==
last_name: string, first_name: string, ssn: string, occupation: string, age: bigint
Project [last_name#596, first_name#597, ssn#598, occupation#599, age#600L]
+- Deduplicate [first_name#597, age#600L, last_name#596, occupation#599, ssn#598]
 +- LogicalRDD [last_name#596, first_name#597, ssn#598, occupation#599, age#600L], false

== Optimized Logical Plan ==
Aggregate [first_name#597, age#600L, last_name#596, occupation#599, ssn#598], [last_name#596, first_name#597, ssn#598, occupation#599, age#600L]
+- LogicalRDD [last_name#596, first_name#597, ssn#598, occupation#599, age#600L], false

== Physical Plan ==
*(2) HashAggregate(keys=[first_name#597, age#600L, last_name#596, occupation#599, ssn#598], functions=[], output=[last_name#596, first_name#597, ssn#598, occupation#599, age#600L])
+- Exchange hashpartitioning(first_name#597, age#600L, last_name#596, occupation#599, ssn#598, 200)
 +- *(1) HashAggregate(keys=[first_name#597, age#600L, last_name#596, occupation#599, ssn#598], functions=[], output=[first_name#597, age#600L, last_name#596, occupation#599, ssn#598])
 +- *(1) Scan ExistingRDD[last_name#596,first_name#597,ssn#598,occupation#599,age#600L]

### (3c): Subtract one from each value using _select_

So far, we've created a distributed DataFrame that is split into many partitions, where each partition is stored on a single machine in our cluster.  Let's look at what happens when we do a basic operation on the dataset.  Many useful data analysis operations can be specified as "do something to each item in the dataset".  These data-parallel operations are convenient because each item in the dataset can be processed individually: the operation on one entry doesn't effect the operations on any of the other entries.  Therefore, Spark can parallelize the operation.

One of the most common DataFrame operations is `select()`, and it works more or less like a SQL `SELECT` statement: You can select specific columns from the DataFrame, and you can even use `select()` to create _new_ columns with values that are derived from existing column values. We can use `select()` to create a new column that decrements the value of the existing `age` column.

`select()` is a _transformation_. It returns a new DataFrame that captures both the previous DataFrame and the operation to add to the query (`select`, in this case). But it does *not* actually execute anything on the cluster. When transforming DataFrames, we are building up a _query plan_. That query plan will be optimized, implemented (in terms of RDDs), and executed by Spark _only_ when we call an action.

In [53]:
# Transform dataDF through a select transformation and rename the newly created '(age -1)' column to 'age'
# Because select is a transformation and Spark uses lazy evaluation, no jobs, stages,
# or tasks will be launched when we run this code.
subDF = dataDF.select('last_name', 'first_name', 'ssn', 'occupation', (dataDF.age - 1).alias('age'))

Let's take a look at the query plan.

In [55]:
subDF.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('last_name, None), unresolvedalias('first_name, None), unresolvedalias('ssn, None), unresolvedalias('occupation, None), (age#600L - 1) AS age#637]
+- AnalysisBarrier
 +- LogicalRDD [last_name#596, first_name#597, ssn#598, occupation#599, age#600L], false

== Analyzed Logical Plan ==
last_name: string, first_name: string, ssn: string, occupation: string, age: bigint
Project [last_name#596, first_name#597, ssn#598, occupation#599, (age#600L - cast(1 as bigint)) AS age#637L]
+- LogicalRDD [last_name#596, first_name#597, ssn#598, occupation#599, age#600L], false

== Optimized Logical Plan ==
Project [last_name#596, first_name#597, ssn#598, occupation#599, (age#600L - 1) AS age#637L]
+- LogicalRDD [last_name#596, first_name#597, ssn#598, occupation#599, age#600L], false

== Physical Plan ==
*(1) Project [last_name#596, first_name#597, ssn#598, occupation#599, (age#600L - 1) AS age#637L]
+- *(1) Scan ExistingRDD[last_name#596,first_name#597,ssn#598,occupation#599,age#600L]

### (3d) Use _collect_ to view results

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3d.png" style="height:700px;float:right"/>

To see a list of elements decremented by one, we need to create a new list on the driver from the the data distributed in the executor nodes.  To do this we can call the `collect()` method on our DataFrame.  `collect()` is often used after transformations to ensure that we are only returning a *small* amount of data to the driver.  This is done because the data returned to the driver must fit into the driver's available memory.  If not, the driver will crash.

The `collect()` method is the first action operation that we have encountered.  Action operations cause Spark to perform the (lazy) transformation operations that are required to compute the values returned by the action.  In our example, this means that tasks will now be launched to perform the `createDataFrame`, `select`, and `collect` operations.

In the diagram, the dataset is broken into four partitions, so four `collect()` tasks are launched. Each task collects the entries in its partition and sends the result to the driver, which creates a list of the values, as shown in the figure below.

Now let's run `collect()` on `subDF`.

In [57]:
# Let's collect the data
results = subDF.collect()
print results

[Row(last_name=u'Brown', first_name=u'Jason', ssn=u'160-37-9051', occupation=u'Aid worker', age=37), Row(last_name=u'Morrison', first_name=u'Earl', ssn=u'361-94-4342', occupation=u'Teacher, music', age=25), Row(last_name=u'Young', first_name=u'Christian', ssn=u'769-27-5887', occupation=u'Sales professional, IT', age=19), Row(last_name=u'George', first_name=u'Daniel', ssn=u'175-24-7915', occupation=u'Geochemist', age=41), Row(last_name=u'Lee', first_name=u'Dawn', ssn=u'310-69-7326', occupation=u'Fine artist', age=24), Row(last_name=u'Hamilton', first_name=u'Matthew', ssn=u'099-90-9730', occupation=u'Best boy', age=42), Row(last_name=u'Amanda', first_name=u'Miss', ssn=u'476-06-5497', occupation=u'Engineer, production', age=41), Row(last_name=u'Fernandez', first_name=u'Thomas', ssn=u'722-09-8354', occupation=u'Psychologist, counselling', age=4), Row(last_name=u'Kirk', first_name=u'Jennifer', ssn=u'715-56-1708', occupation=u'Runner, broadcasting/film/video', age=3), Row(last_name=u'Young', first_name=u'Todd', ssn=u'123-48-8354', occupation=u'Emergency planning/management officer', age=15), Row(last_name=u'Cook', first_name=u'Amy', ssn=u'293-22-0265', occupation=u'Scientist, forensic', age=27), Row(last_name=u'Martinez', first_name=u'Mark', ssn=u'041-23-3263', occupation=u'Broadcast presenter', age=22), Row(last_name=u'Walker', first_name=u'Aaron', ssn=u'725-61-1132', occupation=u'Arts administrator', age=41), Row(last_name=u'Dennis', first_name=u'Edward', ssn=u'268-79-4330', occupation=u'Chief Marketing Officer', age=12), Row(last_name=u'Hebert', first_name=u'Meredith', ssn=u'077-96-8349', occupation=u'Surveyor, land/geomatics', age=13), Row(last_name=u'Hess', first_name=u'Phyllis', ssn=u'061-88-1648', occupation=u'Product manager', age=31), Row(last_name=u'Reed', first_name=u'Kayla', ssn=u'582-28-0099', occupation=u'Maintenance engineer', age=10), Row(last_name=u'Miller', first_name=u'Dwayne', ssn=u'386-07-6013', occupation=u'Set designer', age=4), Row(last_name=u'Weeks', first_name=u'Sean', ssn=u'363-94-7993', occupation=u'Administrator, education', age=40), Row(last_name=u'Williams', first_name=u'Cassie', ssn=u'386-39-5490', occupation=u'Historic buildings inspector/conservation officer', age=9), Row(last_name=u'Rose', first_name=u'Daniel', ssn=u'737-44-0894', occupation=u'Diplomatic Services operational officer', age=5), Row(last_name=u'Dawson', first_name=u'Taylor', ssn=u'790-03-8999', occupation=u'Psychologist, occupational', age=25), Row(last_name=u'Fisher', first_name=u'Steven', ssn=u'789-24-6522', occupation=u'Therapist, drama', age=32), Row(last_name=u'Johnson', first_name=u'Stephanie', ssn=u'702-94-4924', occupation=u'Presenter, broadcasting', age=39), Row(last_name=u'Farmer', first_name=u'Donna', ssn=u'117-61-4564', occupation=u'Museum/gallery curator', age=13), Row(last_name=u'Davis', first_name=u'Tonya', ssn=u'695-93-4517', occupation=u'Communications engineer', age=34), Row(last_name=u'Robert', first_name=u'Dr.', ssn=u'121-52-8368', occupation=u'Buyer, industrial', age=15), Row(last_name=u'Shaw', first_name=u'David', ssn=u'672-57-1650', occupation=u'Production manager', age=22), Row(last_name=u'Paul', first_name=u'Victoria', ssn=u'370-63-8920', occupation=u'Environmental manager', age=20), Row(last_name=u'Hays', first_name=u'Jeff', ssn=u'661-20-4642', occupation=u'Music tutor', age=20), Row(last_name=u'Nguyen', first_name=u'Erica', ssn=u'251-56-0999', occupation=u'Agricultural engineer', age=41), Row(last_name=u'Skinner', first_name=u'Christopher', ssn=u'621-23-3079', occupation=u'Operational investment banker', age=44), Row(last_name=u'Jackson', first_name=u'Nathan', ssn=u'682-09-3573', occupation=u'Administrator, sports', age=33), Row(last_name=u'Thompson', first_name=u'Terri', ssn=u'607-24-8694', occupation=u'Food technologist', age=31), Row(last_name=u'Figueroa', first_name=u'Ashley', ssn=u'439-10-9336', occupation=u'Games developer', age=19), Row(last_name=u'Lewis', first_name=u'David', ssn=u'695-60-0033', occu

A better way to visualize the data is to use the `show()` method. If you don't tell `show()` how many rows to display, it displays 20 rows.

In [59]:
subDF.show()

+---------+----------+-----------+--------------------+---+
last_name|first_name| ssn| occupation|age|
+---------+----------+-----------+--------------------+---+
 Brown| Jason|160-37-9051| Aid worker| 37|
 Morrison| Earl|361-94-4342| Teacher, music| 25|
 Young| Christian|769-27-5887|Sales professiona...| 19|
 George| Daniel|175-24-7915| Geochemist| 41|
 Lee| Dawn|310-69-7326| Fine artist| 24|
 Hamilton| Matthew|099-90-9730| Best boy| 42|
 Amanda| Miss|476-06-5497|Engineer, production| 41|
Fernandez| Thomas|722-09-8354|Psychologist, cou...| 4|
 Kirk| Jennifer|715-56-1708|Runner, broadcast...| 3|
 Young| Todd|123-48-8354|Emergency plannin...| 15|
 Cook| Amy|293-22-0265| Scientist, forensic| 27|
 Martinez| Mark|041-23-3263| Broadcast presenter| 22|
 Walker| Aaron|725-61-1132| Arts administrator| 41|
 Dennis| Edward|268-79-4330|Chief Marketing O...| 12|
 Hebert| Meredith|077-96-8349|Surveyor, land/ge...| 13|
 Hess| Phyllis|061-88-1648| Product manager| 31|
 Reed| Kayla|582-28-0099|Maintenance engineer| 10|
 Miller| Dwayne|386-07-6013| Set designer| 4|
 Weeks| Sean|363-94-7993|Administrator, ed...| 40|
 Williams| Cassie|386-39-5490|Historic building...| 9|
+---------+----------+-----------+--------------------+---+
only showing top 20 rows

If you'd prefer that `show()` not truncate the data, you can tell it not to:

In [61]:
subDF.show(n=30, truncate=False)

+---------+----------+-----------+-------------------------------------------------+---+
last_name|first_name|ssn |occupation |age|
+---------+----------+-----------+-------------------------------------------------+---+
Brown |Jason |160-37-9051|Aid worker |37 |
Morrison |Earl |361-94-4342|Teacher, music |25 |
Young |Christian |769-27-5887|Sales professional, IT |19 |
George |Daniel |175-24-7915|Geochemist |41 |
Lee |Dawn |310-69-7326|Fine artist |24 |
Hamilton |Matthew |099-90-9730|Best boy |42 |
Amanda |Miss |476-06-5497|Engineer, production |41 |
Fernandez|Thomas |722-09-8354|Psychologist, counselling |4 |
Kirk |Jennifer |715-56-1708|Runner, broadcasting/film/video |3 |
Young |Todd |123-48-8354|Emergency planning/management officer |15 |
Cook |Amy |293-22-0265|Scientist, forensic |27 |
Martinez |Mark |041-23-3263|Broadcast presenter |22 |
Walker |Aaron |725-61-1132|Arts administrator |41 |
Dennis |Edward |268-79-4330|Chief Marketing Officer |12 |
Hebert |Meredith |077-96-8349|Surveyor, land/geomatics |13 |
Hess |Phyllis |061-88-1648|Product manager |31 |
Reed |Kayla |582-28-0099|Maintenance engineer |10 |
Miller |Dwayne |386-07-6013|Set designer |4 |
Weeks |Sean |363-94-7993|Administrator, education |40 |
Williams |Cassie |386-39-5490|Historic buildings inspector/conservation officer|9 |
Rose |Daniel |737-44-0894|Diplomatic Services operational officer |5 |
Dawson |Taylor |790-03-8999|Psychologist, occupational |25 |
Fisher |Steven |789-24-6522|Therapist, drama |32 |
Johnson |Stephanie |702-94-4924|Presenter, broadcasting |39 |
Farmer |Donna |117-61-4564|Museum/gallery curator |13 |
Davis |Tonya |695-93-4517|Communications engineer |34 |
Robert |Dr. |121-52-8368|Buyer, industrial |15 |
Shaw |David |672-57-1650|Production manager |22 |
Paul |Victoria |370-63-8920|Environmental manager |20 |
Hays |Jeff |661-20-4642|Music tutor |20 |
+---------+----------+-----------+-------------------------------------------------+---+
only showing top 30 rows

In Databricks, there's an even nicer way to look at the values in a DataFrame: The `display()` helper function.

In [63]:
display(subDF)

last_name,first_name,ssn,occupation,age
Brown,Jason,160-37-9051,Aid worker,37
Morrison,Earl,361-94-4342,"Teacher, music",25
Young,Christian,769-27-5887,"Sales professional, IT",19
George,Daniel,175-24-7915,Geochemist,41
Lee,Dawn,310-69-7326,Fine artist,24
Hamilton,Matthew,099-90-9730,Best boy,42
Amanda,Miss,476-06-5497,"Engineer, production",41
Fernandez,Thomas,722-09-8354,"Psychologist, counselling",4
Kirk,Jennifer,715-56-1708,"Runner, broadcasting/film/video",3
Young,Todd,123-48-8354,Emergency planning/management officer,15


### (3e) Use _count_ to get total

One of the most basic jobs that we can run is the `count()` job which will count the number of elements in a DataFrame, using the `count()` action. Since `select()` creates a new DataFrame with the same number of elements as the starting DataFrame, we expect that applying `count()` to each DataFrame will return the same result.

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3e.png" style="height:700px;float:right"/>

Note that because `count()` is an action operation, if we had not already performed an action with `collect()`, then Spark would now perform the transformation operations when we executed `count()`.

Each task counts the entries in its partition and sends the result to your SparkContext, which adds up all of the counts. The figure on the right shows what would happen if we ran `count()` on a small example dataset with just four partitions.

In [65]:
print dataDF.count()
print subDF.count()

10000
10000

### (3f) Apply transformation _filter_ and view results with _collect_

Next, we'll create a new DataFrame that only contains the people whose ages are less than 10. To do this, we'll use the `filter()` transformation. (You can also use `where()`, an alias for `filter()`, if you prefer something more SQL-like). The `filter()` method is a transformation operation that creates a new DataFrame from the input DataFrame, keeping only values that match the filter expression.

The figure shows how this might work on the small four-partition dataset.

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3f.png" style="height:700px;float:right"/>

To view the filtered list of elements less than 10, we need to create a new list on the driver from the distributed data on the executor nodes.  We use the `collect()` method to return a list that contains all of the elements in this filtered DataFrame to the driver program.

In [67]:
filteredDF = subDF.filter(subDF.age < 10)
filteredDF.show(truncate=False)
filteredDF.count()

+---------+-----------+-----------+-------------------------------------------------+---+
last_name|first_name |ssn |occupation |age|
+---------+-----------+-----------+-------------------------------------------------+---+
Fernandez|Thomas |722-09-8354|Psychologist, counselling |4 |
Kirk |Jennifer |715-56-1708|Runner, broadcasting/film/video |3 |
Miller |Dwayne |386-07-6013|Set designer |4 |
Williams |Cassie |386-39-5490|Historic buildings inspector/conservation officer|9 |
Rose |Daniel |737-44-0894|Diplomatic Services operational officer |5 |
Lewis |David |695-60-0033|Field seismologist |7 |
Rodgers |Beverly |819-57-7194|Records manager |2 |
Rubio |Christopher|750-41-6999|Surveyor, planning and development |9 |
Johnson |Amanda |494-58-8536|Higher education lecturer |2 |
Johnson |Nicholas |157-13-2050|IT consultant |2 |
Johnson |Candice |639-80-6841|Electrical engineer |4 |
Arias |Mark |501-67-7559|Management consultant |2 |
Jones |Jeremy |648-72-8898|Product manager |6 |
Delgado |Sonia |283-58-4951|Solicitor |5 |
Bishop |Jonathan |714-87-5335|Medical physicist |9 |
Pineda |Connor |041-63-6459|Dentist |9 |
Robinson |Adam |279-56-1630|Financial manager |1 |
Holloway |Julia |115-33-9587|Animator |1 |
Fleming |Jodi |833-40-8181|Agricultural engineer |2 |
Powell |Kimberly |102-37-5719|Contracting civil engineer |4 |
+---------+-----------+-----------+-------------------------------------------------+---+
only showing top 20 rows

 Out[ 32 ]: 2287

(These are some _seriously_ precocious children...)

## Part 4: Python Lambda functions and User Defined Functions

Python supports the use of small one-line anonymous functions that are not bound to a name at runtime.

`lambda` functions, borrowed from LISP, can be used wherever function objects are required. They are syntactically restricted to a single expression. Remember that `lambda` functions are a matter of style and using them is never required - semantically, they are just syntactic sugar for a normal function definition. You can always define a separate normal function instead, but using a `lambda` function is an equivalent and more compact form of coding. Ideally you should consider using `lambda` functions where you want to encapsulate non-reusable code without littering your code with one-line functions.

Here, instead of defining a separate function for the `filter()` transformation, we will use an inline `lambda()` function and we will register that lambda as a Spark _User Defined Function_ (UDF). **A UDF is a special wrapper around a function, allowing the function to be used in a DataFrame query.**

In [70]:
from pyspark.sql.types import BooleanType
less_ten = udf(lambda s: s < 10, BooleanType()) # Datatype needs to be defined in lambda
lambdaDF = subDF.filter(less_ten(subDF.age))
lambdaDF.show()
lambdaDF.count()

+---------+-----------+-----------+--------------------+---+
last_name| first_name| ssn| occupation|age|
+---------+-----------+-----------+--------------------+---+
Fernandez| Thomas|722-09-8354|Psychologist, cou...| 4|
 Kirk| Jennifer|715-56-1708|Runner, broadcast...| 3|
 Miller| Dwayne|386-07-6013| Set designer| 4|
 Williams| Cassie|386-39-5490|Historic building...| 9|
 Rose| Daniel|737-44-0894|Diplomatic Servic...| 5|
 Lewis| David|695-60-0033| Field seismologist| 7|
 Rodgers| Beverly|819-57-7194| Records manager| 2|
 Rubio|Christopher|750-41-6999|Surveyor, plannin...| 9|
 Johnson| Amanda|494-58-8536|Higher education ...| 2|
 Johnson| Nicholas|157-13-2050| IT consultant| 2|
 Johnson| Candice|639-80-6841| Electrical engineer| 4|
 Arias| Mark|501-67-7559|Management consul...| 2|
 Jones| Jeremy|648-72-8898| Product manager| 6|
 Delgado| Sonia|283-58-4951| Solicitor| 5|
 Bishop| Jonathan|714-87-5335| Medical physicist| 9|
 Pineda| Connor|041-63-6459| Dentist| 9|
 Robinson| Adam|279-56-1630| Financial manager| 1|
 Holloway| Julia|115-33-9587| Animator| 1|
 Fleming| Jodi|833-40-8181|Agricultural engi...| 2|
 Powell| Kimberly|102-37-5719|Contracting civil...| 4|
+---------+-----------+-----------+--------------------+---+
only showing top 20 rows

 Out[ 33 ]: 2287

In [71]:
# Let's collect the even values less than 10
even = udf(lambda s: s % 2 == 0, BooleanType())
evenDF = lambdaDF.filter(even(lambdaDF.age))
evenDF.show()
evenDF.count()

+---------+----------+-----------+--------------------+---+
last_name|first_name| ssn| occupation|age|
+---------+----------+-----------+--------------------+---+
Fernandez| Thomas|722-09-8354|Psychologist, cou...| 4|
 Miller| Dwayne|386-07-6013| Set designer| 4|
 Rodgers| Beverly|819-57-7194| Records manager| 2|
 Johnson| Amanda|494-58-8536|Higher education ...| 2|
 Johnson| Nicholas|157-13-2050| IT consultant| 2|
 Johnson| Candice|639-80-6841| Electrical engineer| 4|
 Arias| Mark|501-67-7559|Management consul...| 2|
 Jones| Jeremy|648-72-8898| Product manager| 6|
 Fleming| Jodi|833-40-8181|Agricultural engi...| 2|
 Powell| Kimberly|102-37-5719|Contracting civil...| 4|
 Griffith| Valerie|182-03-1388| Exhibition designer| 4|
 Peck| Douglas|133-19-9920| Secretary, company| 0|
 Gallegos| Michael|203-15-1520| Water engineer| 2|
 King|Jacqueline|506-33-4081| Risk manager| 0|
 Hughes| Amy|255-17-3490| Chemical engineer| 8|
 Williams| Nicole|574-72-0358| Advice worker| 2|
 Mack| Bradley|662-80-6804| Market researcher| 2|
 Howard| Nicholas|188-57-7205| Astronomer| 6|
 Smith| Glenn|138-46-4267|English as a fore...| 2|
 Rhodes| Brittany|335-23-7803| Information officer| 6|
+---------+----------+-----------+--------------------+---+
only showing top 20 rows

 Out[ 34 ]: 1081

## Part 5: Additional DataFrame actions

Let's investigate some additional actions:

* [first()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.first)
* [take()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.take)

One useful thing to do when we have a new dataset is to look at the first few entries to obtain a rough idea of what information is available.  In Spark, we can do that using actions like `first()`, `take()`, and `show()`. Note that for the `first()` and `take()` actions, the elements that are returned depend on how the DataFrame is *partitioned*.

Instead of using the `collect()` action, we can use the `take(n)` action to return the first _n_ elements of the DataFrame. The `first()` action returns the first element of a DataFrame, and is equivalent to `take(1)[0]`.

In [73]:
print "first: {0}\n".format(filteredDF.first())

print "Four of them: {0}\n".format(filteredDF.take(4))

first: Row(last_name=u'Fernandez', first_name=u'Thomas', ssn=u'722-09-8354', occupation=u'Psychologist, counselling', age=4)

Four of them: [Row(last_name=u'Fernandez', first_name=u'Thomas', ssn=u'722-09-8354', occupation=u'Psychologist, counselling', age=4), Row(last_name=u'Kirk', first_name=u'Jennifer', ssn=u'715-56-1708', occupation=u'Runner, broadcasting/film/video', age=3), Row(last_name=u'Miller', first_name=u'Dwayne', ssn=u'386-07-6013', occupation=u'Set designer', age=4), Row(last_name=u'Williams', first_name=u'Cassie', ssn=u'386-39-5490', occupation=u'Historic buildings inspector/conservation officer', age=9)]

This looks better:

In [75]:
display(filteredDF.take(4))

last_name,first_name,ssn,occupation,age
Fernandez,Thomas,722-09-8354,"Psychologist, counselling",4
Kirk,Jennifer,715-56-1708,"Runner, broadcasting/film/video",3
Miller,Dwayne,386-07-6013,Set designer,4
Williams,Cassie,386-39-5490,Historic buildings inspector/conservation officer,9


## Part 6: Additional DataFrame transformations

### (6a) _orderBy_

[`orderBy()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) allows you to sort a DataFrame by one or more columns, producing a new DataFrame.

For example, let's get the first five oldest people in the original (unfiltered) DataFrame. We can use the `orderBy()` transformation. `orderBy` takes one or more columns, either as _names_ (strings) or as `Column` objects. To get a `Column` object, we use one of two notations on the DataFrame:

* Pandas-style notation: `filteredDF.age`
* Subscript notation: `filteredDF['age']`

Both of those syntaxes return a `Column`, which has additional methods like `desc()` (for sorting in descending order) or `asc()` (for sorting in ascending order, which is the default).

Here are some examples:

```
dataDF.orderBy(dataDF['age'])  # sort by age in ascending order; returns a new DataFrame
dataDF.orderBy(dataDF.last_name.desc()) # sort by last name in descending order
```

In [78]:
# Get the five oldest people in the list. To do that, sort by age in descending order.
display(dataDF.orderBy(dataDF.age.desc()).take(5))

last_name,first_name,ssn,occupation,age
Harper,Robert,332-72-3464,"Lighting technician, broadcasting/film/video",47
Larson,Catherine,513-60-3149,Multimedia specialist,47
Lee,Pamela,272-14-8272,"Engineer, automotive",47
Parker,Jason,151-03-4564,Environmental manager,47
Caldwell,Emily,555-90-6944,Accounting technician,47


Let's reverse the sort order. Since ascending sort is the default, we can actually use a `Column` object expression or a simple string, in this case. **The `desc()` and `asc()` methods are only defined on `Column`. Something like `orderBy('age'.desc())` would not work**, because there's no `desc()` method on Python string objects. That's why we needed the column expression. But if we're just using the defaults, we can pass a string column name into `orderBy()`. This is sometimes easier to read.

In [80]:
display(dataDF.orderBy('age').take(5))

last_name,first_name,ssn,occupation,age
Myers,Donna,844-84-7394,Early years teacher,1
Munoz,Laura,776-13-4126,Ship broker,1
Rivera,Katelyn,445-27-3293,Child psychotherapist,1
Schmidt,Maria,760-26-2009,Building services engineer,1
Singleton,Jennifer,652-23-8827,Tax adviser,1


### (6b) _distinct_ and _dropDuplicates_

[`distinct()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) filters out duplicate rows, and it considers all columns. Since our data is completely randomly generated (by `fake-factory`), it's extremely unlikely that there are any duplicate rows:

In [82]:
print dataDF.count()
print dataDF.distinct().count()

10000
10000

To demonstrate `distinct()`, let's create a quick throwaway dataset.

In [84]:
tempDF = sqlContext.createDataFrame([("Joe", 1), ("Joe", 1), ("Anna", 15), ("Anna", 12), ("Ravi", 5)], ('name', 'score'))

In [85]:
tempDF.show()

+----+-----+
name|score|
+----+-----+
 Joe| 1|
 Joe| 1|
Anna| 15|
Anna| 12|
Ravi| 5|
+----+-----+

In [86]:
tempDF.distinct().show()

+----+-----+
name|score|
+----+-----+
 Joe| 1|
Ravi| 5|
Anna| 12|
Anna| 15|
+----+-----+

Note that one of the ("Joe", 1) rows was deleted, but both rows with name "Anna" were kept, because all columns in a row must match another row for it to be considered a duplicate.

[`dropDuplicates()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates) is like `distinct()`, except that `dropDuplicates()` allows us to **specify the columns to compare**. For instance, we can use it to drop all rows where the first name and last name duplicates (ignoring the occupation and age columns).

In [89]:
print dataDF.count()
print dataDF.dropDuplicates(['first_name', 'last_name']).count()

10000
9347

### (6c) _drop_

[`drop()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop) is like the opposite of `select()`: Instead of selecting specific columns from a DataFrame, it drops a specifed column from a DataFrame.

Here's a simple use case: Suppose you're reading from a 1,000-column CSV file, and you have to get rid of five of the columns. Instead of selecting 995 of the columns, it's easier just to drop the five you don't want.

In [91]:
dataDF.drop('occupation').drop('age').show()

+---------+----------+-----------+
last_name|first_name| ssn|
+---------+----------+-----------+
 Brown| Jason|160-37-9051|
 Morrison| Earl|361-94-4342|
 Young| Christian|769-27-5887|
 George| Daniel|175-24-7915|
 Lee| Dawn|310-69-7326|
 Hamilton| Matthew|099-90-9730|
 Amanda| Miss|476-06-5497|
Fernandez| Thomas|722-09-8354|
 Kirk| Jennifer|715-56-1708|
 Young| Todd|123-48-8354|
 Cook| Amy|293-22-0265|
 Martinez| Mark|041-23-3263|
 Walker| Aaron|725-61-1132|
 Dennis| Edward|268-79-4330|
 Hebert| Meredith|077-96-8349|
 Hess| Phyllis|061-88-1648|
 Reed| Kayla|582-28-0099|
 Miller| Dwayne|386-07-6013|
 Weeks| Sean|363-94-7993|
 Williams| Cassie|386-39-5490|
+---------+----------+-----------+
only showing top 20 rows

### (6d) _groupBy_

[`groupBy()`]((http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) is **one of the most powerful transformations**. It allows you to **perform aggregations on a DataFrame**.

Unlike other DataFrame transformations, `groupBy()` does _not_ return a DataFrame. Instead, it returns a special [GroupedData](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) object that contains various aggregation functions.

**The most commonly used aggregation function is [count()]**(http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.count),
but there are others (like [sum()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.sum), [max()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.max), and [avg()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.avg).

These aggregation functions typically create a new column and return a new DataFrame.

In [93]:
dataDF.groupBy('occupation').count().show(truncate=False)

+---------------------------------------+-----+
occupation |count|
+---------------------------------------+-----+
Diplomatic Services operational officer|16 |
Librarian, academic |18 |
Catering manager |23 |
Retail merchandiser |14 |
Designer, ceramics/pottery |18 |
Engineer, aeronautical |11 |
English as a second language teacher |20 |
Early years teacher |17 |
Occupational hygienist |16 |
Patent examiner |12 |
Primary school teacher |14 |
Clinical molecular geneticist |17 |
Control and instrumentation engineer |18 |
Estate agent |19 |
Art therapist |13 |
Transport planner |21 |
Applications developer |19 |
Petroleum engineer |23 |
Adult nurse |24 |
Conservator, furniture |9 |
+---------------------------------------+-----+
only showing top 20 rows

In [94]:
dataDF.groupBy().avg('age').show(truncate=False)

+--------+
avg(age)|
+--------+
23.4683 |
+--------+

We can also use `groupBy()` to do aother useful aggregations:

In [96]:
print "Maximum age: {0}".format(dataDF.groupBy().max('age').first()[0])
print "Minimum age: {0}".format(dataDF.groupBy().min('age').first()[0])

Maximum age: 47
Minimum age: 1

Show the schema of dataDF

In [98]:
dataDF.schema

Out[ 48 ]: StructType(List(StructField(last_name,StringType,true),StructField(first_name,StringType,true),StructField(ssn,StringType,true),StructField(occupation,StringType,true),StructField(age,LongType,true)))

### (6e) _sample_ (optional)

When analyzing data, the [`sample()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample) transformation is often quite useful. It returns a new DataFrame with a random sample of elements from the dataset.  It takes in a `withReplacement` argument, which specifies whether it is okay to randomly pick the same item multiple times from the parent DataFrame (so when `withReplacement=True`, you can get the same item back multiple times). It takes in a `fraction` parameter, which specifies the fraction elements in the dataset you want to return. (So a `fraction` value of `0.20` returns 20% of the elements in the DataFrame.) It also takes an optional `seed` parameter that allows you to specify a seed value for the random number generator, so that reproducible results can be obtained.

In [100]:
sampledDF = dataDF.sample(withReplacement=False, fraction=0.10)
print sampledDF.count()
sampledDF.show()
# The result is not exactly 10000*0.1 = 1000, but 943 (1st time), 976 (2nd time), 1011 (3rd time) ...

944
+---------+----------+-----------+--------------------+---+
last_name|first_name| ssn| occupation|age|
+---------+----------+-----------+--------------------+---+
 Robert| Dr.|121-52-8368| Buyer, industrial| 16|
 Hall| Monique|518-71-6777|Research scientis...| 38|
 Johnson| Nicholas|157-13-2050| IT consultant| 3|
 Walker| Joseph|079-99-5775|Restaurant manage...| 29|
 Fleming| Jodi|833-40-8181|Agricultural engi...| 3|
 Williams| Brandon|590-16-3719|Doctor, general p...| 16|
 Hunt| Michael|745-13-3849|Rural practice su...| 24|
 Smith| Rachel|708-44-5175|Surveyor, buildin...| 38|
 Murphy| Michael|663-26-5493| Surveyor, mining| 6|
 Davis| Briana|687-12-3800|Research officer,...| 24|
 Johnson| Hector|753-83-1027|Surveyor, rural p...| 16|
 Moore| Scott|514-35-9041|Public house manager| 24|
 Evans| Raven|829-56-8556| Health physicist| 8|
 Thornton| Heather|265-74-7056| Solicitor| 33|
 Riggs| Brianna|320-57-4634|Licensed conveyancer| 30|
 Smith| Kylie|788-44-1498|Housing manager/o...| 33|
 Shaw| Jill|131-34-6650|Television produc...| 23|
 Ellis| Steven|058-86-5174| Purchasing manager| 4|
 Jimenez| Elizabeth|216-68-7534|Furniture conserv...| 40|
 Bryant| Danielle|686-64-7983| Proofreader| 24|
+---------+----------+-----------+--------------------+---+
only showing top 20 rows

In [101]:
print dataDF.sample(withReplacement=False, fraction=0.05).count()

505

## Part 7: Caching DataFrames and storage options

### (7a) Caching DataFrames

For efficiency Spark keeps your DataFrames in memory. (More formally, it keeps the _RDDs_ that implement your DataFrames in memory.) By keeping the contents in memory, Spark can quickly access the data. **However, memory is limited, so if you try to keep too many partitions in memory, Spark will automatically delete partitions from memory to make space for new ones.** If you later refer to one of the deleted partitions, Spark will automatically recreate it for you, but that takes time.

So, **if you plan to use a DataFrame more than once, then you should tell Spark to cache it. You can use the `cache()` operation to keep the DataFrame in memory.** However, **you must still trigger an action on the DataFrame, such as `collect()` or `count()` before the caching will occur.** In other words, `cache()` is lazy: It merely tells Spark that the DataFrame should be cached _when the data is materialized_. You have to run an action to materialize the data; the DataFrame will be cached as a side effect. The next time you use the DataFrame, Spark will use the cached data, rather than recomputing the DataFrame from the original data.

You can see your cached DataFrame in the "Storage" section of the Spark web UI. If you click on the name value, you can see more information about where the the DataFrame is stored.

In [104]:
# Cache the DataFrame
filteredDF.cache()
# Trigger an action
print filteredDF.count()
# Check if it is cached
print filteredDF.is_cached

2287
True

### (7b) Unpersist and storage options

Spark automatically manages the partitions cached in memory. If it has more partitions than available memory, by default, it will evict older partitions to make room for new ones. For efficiency, once you are finished using cached DataFrame, you can optionally tell Spark to stop caching it in memory by using the DataFrame's `unpersist()` method to inform Spark that you no longer need the cached data.

** Advanced: ** Spark provides many more options for managing how DataFrames cached. For instance, you can tell Spark to spill cached partitions to disk when it runs out of memory, instead of simply throwing old ones away. You can explore the API for DataFrame's [persist()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.persist) operation using Python's [help()](https://docs.python.org/2/library/functions.html?highlight=help#help) command.  The `persist()` operation, optionally, takes a pySpark [StorageLevel](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.StorageLevel) object.

In [106]:
# If we are done with the DataFrame we can unpersist it so that its memory can be reclaimed
filteredDF.unpersist()
# Check if it is cached
print filteredDF.is_cached

False

## ** Part 8: Debugging Spark applications and lazy evaluation **

### How Python is Executed in Spark

Internally, Spark executes using a Java Virtual Machine (JVM). **pySpark runs Python code in a JVM using [Py4J]**(http://py4j.sourceforge.net). Py4J enables Python programs running in a Python interpreter to dynamically access Java objects in a Java Virtual Machine. Methods are called as if the Java objects resided in the Python interpreter and Java collections can be accessed through standard Python collection methods. Py4J also enables Java programs to call back Python objects.

Because pySpark uses Py4J, coding errors often result in a complicated, confusing stack trace that can be difficult to understand. In the following section, we'll explore how to understand stack traces.

### (8a) Challenges with lazy evaluation using transformations and actions

Spark's use of lazy evaluation can make debugging more difficult because code is not always executed immediately. To see an example of how this can happen, let's first define a broken filter function.
Next we perform a `filter()` operation using the broken filtering function.  No error will occur at this point due to Spark's use of lazy evaluation.

The `filter()` method will not be executed *until* an action operation is invoked on the DataFrame.  We will perform an action by using the `count()` method to return a list that contains all of the elements in this DataFrame.

In [110]:
def brokenTen(value):
    """Incorrect implementation of the ten function.

    Note:
        The `if` statement checks an undefined variable `val` instead of `value`.

    Args:
        value (int): A number.

    Returns:
        bool: Whether `value` is less than ten.

    Raises:
        NameError: The function references `val`, which is not available in the local or global
            namespace, so a `NameError` is raised.
    """
    if (val < 10):
        return True
    else:
        return False

btUDF = udf(brokenTen)
brokenDF = subDF.filter(btUDF(subDF.age) == True)

In [111]:
# Now we'll see the error
# Click on the `+` button to expand the error and scroll through the message.
brokenDF.count()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2783183391559683> in <module> () 
 1 # Now we'll see the error 
 2 # Click on the `+` button to expand the error and scroll through the message. 
 ----> 3 brokenDF . count ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.pyc in count (self) 
 455 2 
 456 """
 --> 457 return int ( self . _jdf . count ( ) ) 
 458 
 459 @ ignore_unicode_prefix 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.pyc in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o879.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 298.0 failed 1 times, most recent failure: Lost task 7.0 in stage 298.0 (TID 4234, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 262, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 257, in process
 serializer.dump_stream(func(split_index, iterator), outfile)
 File "/databricks/spark/python/pyspark/worker.py", line 183, in <lambda>
 func = lambda _, it: map(mapper, it)
 File "<string>", line 1, in <lambda>
 File "/databricks/spark/python/pyspark/worker.py", line 79, in <lambda>
 return lambda *a: f(*a)
 File "/databricks/spark/python/pyspark/util.py", line 55, in wrapper
 return f(*args, **kwargs)
 File "<command-2783183391559682>", line 17, in brokenTen
NameError: global name 'val' is not defined

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:317)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:83)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:66)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:271)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:439)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.agg_doAggregateWithoutKey_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:620)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:125)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:96)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:53)
	at org.apache.spark.scheduler.Task.run(Task.scala:112)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:384)
	at java.util.concurrent.ThreadPoolExecutor.runW

### (8b) Finding the bug

When the `filter()` method is executed, Spark calls the UDF. Since our UDF has an error in the underlying filtering function `brokenTen()`, an error occurs.

Scroll through the output "Py4JJavaError     Traceback (most recent call last)" part of the cell and first you will see that the line that generated the error is the `count()` method line. There is *nothing wrong with this line*. However, it is an action and that caused other methods to be executed. Continue scrolling through the Traceback and you will see the following error line:

`NameError: global name 'val' is not defined`

Looking at this error line, we can see that we used the wrong variable name in our filtering function `brokenTen()`.

### (8c) Moving toward expert style

**As you are learning Spark, I recommend that you write your code in the form**:
```
    df2 = df1.transformation1()
    df2.action1()
    df3 = df2.transformation2()
    df3.action2()
```
Using this style will make debugging your code much easier as it makes errors easier to localize - errors in your transformations will occur when the next action is executed.

Once you become more experienced with Spark, you can write your code with the form: `df.transformation1().transformation2().action()`

We can also use `lambda()` functions instead of separately defined functions when their use improves readability and conciseness.

In [114]:
# Cleaner code through lambda use
myUDF = udf(lambda v: v < 10)
subDF.filter(myUDF(subDF.age) == True)

### (8d) Readability and code style

To make the expert coding style more readable, enclose the statement in parentheses and **put each method, transformation, or action on a separate line**.

In [116]:
# Final version
from pyspark.sql.functions import *
(dataDF
 .filter(dataDF.age > 20)
 .select(concat(dataDF.first_name, lit(' '), dataDF.last_name), dataDF.occupation)
 .show(truncate=False)
 )